In [29]:
import pandas as pd
import sys
from itertools import combinations
from crossref.restful import Works
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
import gensim
from gensim import corpora
import itertools
import time
from pyvis.network import Network
sys.path.append("..")
from rcn_py import orcid
from rcn_py import scopus

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Just trying to see if the package is available
orcid_id = orcid.name_to_orcid_id("Stefan Verhoeven")
print(orcid_id)

0000-0002-5821-2060


In [17]:
orcid_record = orcid.query_orcid_for_record(orcid_id)

In [92]:
orcid_record['person']['addresses']['address'][0]['country']['value']

'NL'

In [9]:
orcid_record['person']['name']['given-names']['value']+' '+orcid_record['person']['name']['family-name']['value']

'Stefan Verhoeven'

In [42]:
works = orcid.extract_works_section(orcid_record)
dois = []
for work in works:
    doi,title = orcid.extract_doi(work)
    dois.append(doi)

In [20]:
orcid_record['person']

{'last-modified-date': {'value': 1648113271690},
 'name': {'created-date': {'value': 1467114399264},
  'last-modified-date': {'value': 1467114399264},
  'given-names': {'value': 'Stefan'},
  'family-name': {'value': 'Verhoeven'},
  'credit-name': None,
  'source': None,
  'visibility': 'public',
  'path': '0000-0002-5821-2060'},
 'other-names': {'last-modified-date': None,
  'other-name': [],
  'path': '/0000-0002-5821-2060/other-names'},
 'biography': None,
 'researcher-urls': {'last-modified-date': None,
  'researcher-url': [],
  'path': '/0000-0002-5821-2060/researcher-urls'},
 'emails': {'last-modified-date': {'value': 1648113271690},
  'email': [{'created-date': {'value': 1467114763304},
    'last-modified-date': {'value': 1648113271690},
    'source': {'source-orcid': {'uri': 'https://orcid.org/0000-0002-5821-2060',
      'path': '0000-0002-5821-2060',
      'host': 'orcid.org'},
     'source-client-id': None,
     'source-name': {'value': 'Stefan Verhoeven'},
     'assertion-ori

In [36]:
works[0]['work-summary'][0].keys()

dict_keys(['put-code', 'created-date', 'last-modified-date', 'source', 'title', 'external-ids', 'url', 'type', 'publication-date', 'journal-title', 'visibility', 'path', 'display-index'])

In [32]:
def get_authors(doi):
    works = Works()
    metadata = works.doi(doi)
    # It's possible that the acquired  
    if (metadata) and ('author' in metadata.keys()):
        authors = metadata['author']
        orcid_list = []
        author_names = []

        for i in authors:
            if 'ORCID' in i.keys():
                orcid_id_split = i['ORCID'].split("http://orcid.org/")[1]
                orcid_list.append(orcid_id_split)
            elif 'given' in i.keys() and 'family' in i.keys():
                orcid_id_by_name = orcid.name_to_orcid_id(i['given']+' '+i['family'])
                orcid_list.append(orcid_id_by_name)
            else:
                continue
            author_names.append(i['given']+' '+i['family'])
        return orcid_list, author_names
    else:
        return [],[]


In [33]:
a,b=get_authors("10.5194/egusphere-egu2020-11730")

In [53]:
works = Works()
metadata = works.doi('10.7717/peerj.8214',)

In [54]:
metadata['title']

['sv-callers: a highly portable parallel workflow for structural variant detection in whole-genome sequence data']

In [55]:
metadata['abstract']

'<jats:p>Structural variants (SVs) are an important class of genetic variation implicated in a wide array of genetic diseases including cancer. Despite the advances in whole genome sequencing, comprehensive and accurate detection of SVs in short-read data still poses some practical and computational challenges. We present <jats:italic>sv-callers</jats:italic>, a highly portable workflow that enables parallel execution of multiple SV detection tools, as well as provide users with example analyses of detected SV callsets in a Jupyter Notebook. This workflow supports easy deployment of software dependencies, configuration and addition of new analysis tools. Moreover, porting it to different computing systems requires minimal effort. Finally, we demonstrate the utility of the workflow by performing both somatic and germline SV analyses on different high-performance computing systems.</jats:p>'

In [14]:
all_orcid = []
all_names = []
for work in works:
    doi,title = orcid.extract_doi(work)
    orcid_list, name_list = orcid.get_authors(doi)
    all_orcid += orcid_list
    all_names += name_list

In [61]:
clean_text(metadata['title'][0])

['sv',
 'caller',
 'highli',
 'portabl',
 'parallel',
 'workflow',
 'structur',
 'variant',
 'detect',
 'whole',
 'genom',
 'sequenc',
 'data']

In [2]:
def clean_text(text):
    text = text.replace('\n'," ") 
    text = re.sub(r"-", " ", text) 
    text = re.sub(r"\d+/\d+/\d+", "", text) 
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text) 
    text = re.sub(r"[\w]+@[\.\w]+", "", text) 
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text)
    pure_text = ''
    for letter in text:
        # Leave only letters and spaces
        if letter.isalpha() or letter==' ':
            letter = letter.lower()
            pure_text += letter
            
    corpus_lst = [ps.stem(word) for word in pure_text.split() if word not in stop_words]
    return corpus_lst

In [3]:
def orcid_lda_cluster(dois):
    abstracts = []
    cleaned_abs_corpus = []
    clusters = {}
    works = Works()
    for i in dois:
        w = works.doi(i)
        if 'abstract' in w.keys():
            cleaned_abs_corpus.append(clean_text(w['abstract']))
        elif 'title' in w.keys() and w['title']:
            cleaned_abs_corpus.append(clean_text(w['title'][0]))
        else:
            cleaned_abs_corpus.append([])

    num_topics = 10
    dictionary = corpora.Dictionary(cleaned_abs_corpus) 
    corpus = [dictionary.doc2bow(text) for text in cleaned_abs_corpus]

    t0= time.time()
    passes= 150
    np.random.seed(1)

    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics,
                                           chunksize= 4000, 
                                           batch= True,
                                           minimum_probability=0.001,
                                           iterations=350,
                                           passes=passes)
    group_num = []
    for i in range(len(cleaned_abs_corpus)):
        scores = []
        for j1,j2 in lda_model[corpus[i]]:
            scores.append(j2)
            clusters[dois[i]] = scores.index(max(scores))
        group_num.append(scores.index(max(scores)))
        
    return clusters

In [74]:
clusters = orcid_lda_cluster(dois)

In [101]:
a = np.array([0, 1, 2, 5, 5, 5, 1, 5, 1, 0, 8, 6, 0, 9, 6, 0, 0, 0])

b=np.argsort(a)
print(b)

[ 0 15 12  9 16 17  6  1  8  2  7  4  3  5 11 14 10 13]


In [102]:
group_temp = []
b = []
for g in a:
    if g not in group_temp:
        group_temp.append(g)
    b.append(group_temp.index(g))
b

[0, 1, 2, 3, 3, 3, 1, 3, 1, 0, 4, 5, 0, 6, 5, 0, 0, 0]

In [11]:
def orcid_get_coauthors(full_name):
    orcid_id = orcid.name_to_orcid_id(full_name)
    orcid_record = orcid.query_orcid_for_record(orcid_id)
    docs = orcid.extract_works_section(orcid_record)
    
    all_orcid = []
    all_names = []
    all_group = []
    coauthor_links = []

    # LDA topic modeling
    doi_list = []
    for doc in docs:
        doi, title = orcid.extract_doi(doc)
        doi_list.append(doi)
    cluster_dict = orcid_lda_cluster(doi_list)
        
    for doc in docs:
        doi, title = orcid.extract_doi(doc)
        doc_group = cluster_dict[doi]
        orcid_list, name_list = orcid.get_authors(doi)
        # Geo-filtering 
        filtered_orcid_list = []
        filtered_name_list = []
        author_group_list = []
        for i in range(len(orcid_list)):
            # orcid_record = orcid.query_orcid_for_record(orcid_list[i])
            # if orcid_record['person']['addresses']['address']:
            #     country = orcid_record['person']['addresses']['address'][0]['country']['value']
            # else:
            #     country = ''
            # if country == 'NL':
                filtered_orcid_list.append(orcid_list[i])
                filtered_name_list.append(name_list[i])
                author_group_list.append(doc_group)

        # Combination
        if len(filtered_orcid_list) >=2:
            coauthor_links = coauthor_links+list(combinations(filtered_orcid_list, 2))

        all_orcid += filtered_orcid_list
        all_names += filtered_name_list
        all_group += author_group_list

    group_temp = []
    new_group_num = []
    for g in all_group:
        if g not in group_temp:
            group_temp.append(g)
        new_group_num.append(group_temp.index(g))
        
    df = pd.DataFrame()
    df['orcid'] = all_orcid
    df['name'] = all_names
    df['group'] = new_group_num

    new_df = df.drop_duplicates(subset = ['orcid'],keep='first', ignore_index=True)
    
    return new_df, coauthor_links
    

In [17]:
my_df = orcid.get_coauthors("Stefan Verhoeven")

In [12]:
df1, link1 = orcid_get_coauthors("Peter Kalverla")

In [18]:
my_df

,orcid,name
0,0000-0001-9795-7981,Niels Drost
1,0000-0002-5697-5697,Rolf Hut
2,0000-0002-7200-3353,Nick Van De Giesen
3,0000-0002-7508-3272,Ben van Werkhoven
4,0000-0003-0157-4818,Jerom P.M. Aerts
...,...,...
84,0000-0001-6859-4421,Blaise TF Alako
85,0000-0001-6041-844X,Antoine Veldhoven
86,0000-0001-5351-4361,Sjozef van Baal
87,0000-0002-6395-1407,Rob Jelier


In [119]:
test_list = []
for i in my_df['orcid']:
    orcid_record = orcid.query_orcid_for_record(i)
    if orcid_record['person']['addresses']['address']:
        country = orcid_record['person']['addresses']['address'][0]['country']['value']
    else:
        country = ''
    if country == 'NL':
        test_list.append(i)
len(test_list)


19

In [13]:
df1

,orcid,name,group
0,0000-0002-3986-1268,Ruth Lorenz,0
1,0000-0001-5760-4524,Lukas Brunner,0
2,0000-0002-5025-7862,Peter Kalverla,0
3,0000-0002-5413-9038,Stef Smeets,0
4,0000-0002-8928-7831,Jaro Camphuijsen,0
5,0000-0001-9005-8940,Bouwe Andela,0
6,0000-0001-8407-6472,Fakhereh Alidoost,0
7,0000-0003-0157-4818,Jerom Aerts,0
8,0000-0002-7200-3353,Nick van De Giesen,0
9,0000-0001-8367-1333,Gijs van Den Oord,0


In [25]:
g2 = df1.loc[(df1['orcid']=='0000-0001-6734-2660')]['group']     

In [26]:
g2+1

38    3
Name: group, dtype: int64

In [15]:
link1

[('0000-0002-3986-1268', '0000-0001-5760-4524'),
 ('0000-0002-3986-1268', '0000-0002-5025-7862'),
 ('0000-0002-3986-1268', '0000-0002-5413-9038'),
 ('0000-0002-3986-1268', '0000-0002-8928-7831'),
 ('0000-0002-3986-1268', '0000-0001-9005-8940'),
 ('0000-0001-5760-4524', '0000-0002-5025-7862'),
 ('0000-0001-5760-4524', '0000-0002-5413-9038'),
 ('0000-0001-5760-4524', '0000-0002-8928-7831'),
 ('0000-0001-5760-4524', '0000-0001-9005-8940'),
 ('0000-0002-5025-7862', '0000-0002-5413-9038'),
 ('0000-0002-5025-7862', '0000-0002-8928-7831'),
 ('0000-0002-5025-7862', '0000-0001-9005-8940'),
 ('0000-0002-5413-9038', '0000-0002-8928-7831'),
 ('0000-0002-5413-9038', '0000-0001-9005-8940'),
 ('0000-0002-8928-7831', '0000-0001-9005-8940'),
 ('0000-0001-8407-6472', '0000-0003-0157-4818'),
 ('0000-0001-8407-6472', '0000-0001-9005-8940'),
 ('0000-0001-8407-6472', '0000-0002-8928-7831'),
 ('0000-0001-8407-6472', '0000-0002-7200-3353'),
 ('0000-0001-8407-6472', '0000-0001-8367-1333'),
 ('0000-0001-8407-64

In [49]:
def orcid_network(author_df, link):
    
    sources = []
    targets = []
    weights = []

    for i in link:
        sources.append(i[0])
        targets.append(i[1])
        weights.append(link.count(i))
    
    # Pyvis network
    N = Network(height=800, width="100%", bgcolor="#222222", font_color="white")
    N.toggle_hide_edges_on_drag(False)
    N.barnes_hut()

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        N.add_node(src, src, title=src )
        N.add_node(dst, dst, title=dst)
        N.add_edge(src, dst, value=w)


    # add neighbor data to node hover data
    for node in N.nodes:
        node["title"] = node["id"]
        node["label"] = node['id']
       

    N.show("new39.html")

In [50]:
orcid_network(df1, link1)

In [38]:
sources = []
targets = []
weights = []

for i in link1:
        sources.append(i[0])
        targets.append(i[1])
        weights.append(link1.count(i))


In [48]:
N = Network(height=800, width="100%", bgcolor="#222222", font_color="white", select_menu=True)
N.toggle_hide_edges_on_drag(False)
N.barnes_hut()

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    N.add_node(src, src, title=src)
    N.add_node(dst, dst, title=dst)
    N.add_edge(src, dst, value=w)


# add neighbor data to node hover data
for node in N.nodes:
    node["title"] = node["id"]
    node["label"] = node["id"]

N.show("new39.html")